In [82]:
from google.cloud import bigquery
from functions import *
# Numpy & Pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

import warnings

# Set the warning filter to ignore all warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector

from scipy.stats import chi2_contingency

In [2]:
# set display options to show all columns
pd.set_option('display.max_columns', None)
# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Set the client for future queries to BigQuery
client = bigquery.Client(project = "continente-lced-feup")
#data_table.enable_dataframe_formatter()
#auth.authenticate_user()

In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=6coJ3w3JGSZ2bDNRcpNw4XYLkEJpZZ&access_type=offline&code_challenge=vhoUBKB4eVOC3PGt0OgtoM2nUqyXTx0LBCDoDwLGltE&code_challenge_method=S256


Credentials saved to file: [/Users/vp/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "continente-lced-feup" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the r

In [4]:
query = client.query("""
   SELECT *
   FROM 
       tables_staging.df_models
  LIMIT 2000000
   """)

df = query.result().to_dataframe() # Wait for the job to complete.
df

,CUSTOMER_ACCOUNT_NR_MASK,SUBCAT_CD_EXT,MONTH,YEAR,QUARTER,SEMESTER,CUST_NUM_TRANSACTIONS_30_DAYS,CUST_NUM_TRANSACTIONS_90_DAYS,CUST_NUM_TRANSACTIONS_180_DAYS,CUST_NUM_TRANSACTIONS_360_DAYS,CUST_TOTAL_QTY_BOUGHT_30_DAYS,CUST_TOTAL_QTY_BOUGHT_90_DAYS,CUST_TOTAL_QTY_BOUGHT_180_DAYS,CUST_TOTAL_QTY_BOUGHT_360_DAYS,CUST_NUM_UNIQUE_SUBCAT_30_DAYS,CUST_NUM_UNIQUE_SUBCAT_90_DAYS,CUST_NUM_UNIQUE_SUBCAT_180_DAYS,CUST_NUM_UNIQUE_SUBCAT_360_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS,REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUST_AVG_BASKET_SIZE_30_DAYS,CUST_AVG_BASKET_SIZE_90_DAYS,CUST_AVG_BASKET_SIZE_180_DAYS,CUST_AVG_BASKET_SIZE_360_DAYS,SUBCAT_NUM_TRANSACTIONS_30_DAYS,SUBCAT_NUM_TRANSACTIONS_90_DAYS,SUBCAT_NUM_TRANSACTIONS_180_DAYS,SUBCAT_NUM_TRANSACTIONS_360_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_30_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_90_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_180_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_360_DAYS,SUBCAT_NUM_UNIQUE_CUST_30_DAYS,SUBCAT_NUM_UNIQUE_CUST_90_DAYS,SUBCAT_NUM_UNIQUE_CUST_180_DAYS,SUBCAT_NUM_UNIQUE_CUST_360_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_30_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_30_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_90_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_180_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_360_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD,CAT_CD_EXT,TARGET
0,9555110,60102,1,2022,1,1,2,5,13,25,38,66,226,326,19,28,56,66,15,17,14,14,6.00,11.00,9.00,12.08,9.40,117,282,524,994,1245,3069,6024,11265,981,1876,2710,3487,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",1,4,601,0
1,104579603,60102,1,2022,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,117,282,524,994,1245,3069,6024,11265,981,1876,2710,3487,0,0,0,0,0,0,0,0,0,0,0,0,M,"(1, 2)",2,5,601,0
2,1715871,60102,1,2022,1,1,6,8,9,30,70,94,106,293,29,37,40,56,5,10,13,12,6.00,9.00,9.38,9.67,7.27,117,282,524,994,1245,3069,6024,11265,981,1876,2710,3487,0,0,0,0,0,0,0,0,0,0,0,0,F,"(1, 2)",2,4,601,1
3,50658166,60102,1,2022,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.00,0.00,0.00,0.00,0.00,117,282,524,994,1245,3069,6024,11265,981,1876,2710,3487,0,0,0,0,0,0,0,0,0,0,0,0,M,"(1, 2)",3,3,601,0
4,2937798,140301,1,2022,1,1,1,7,15,27,15,156,339,634,9,35,46,53,30,14,12,13,13.00,9.00,10.71,11.67,11.11,47,144,267,515,1744,4955,9330,18689,979,1873,2534,3439,0,0,0,0,0,0,0,0,0,0,0,0,M,"(1, 2)",3,5,1403,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,102696983,30203,11,2022,4,2,6,22,24,25,94,247,251,252,37,57,57,58,5,4,4,14,5.00,10.50,8.09,7.54,7.28,152,477,1039,1788,1323,4145,8832,17031,660,1422,2336,3203,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",3,5,302,0
1999996,19276403,30301,12,2021,4,2,1,10,12,19,3,23,28,40,3,14,16,24,30,9,11,17,0.00,3.00,2.00,2.00,1.89,238,643,1453,2927,3508,7775,16614,32391,908,1691,2757,3681,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",3,4,303,0
1999997,66629186,80111,3,2021,1,1,2,10,10,10,8,29,29,29,4,8,8,8,24,7,7,7,0.00,3.00,2.30,2.30,2.30,3,8,8,8,28,136,136,136,14,50,50,50,0,0,0,0,0,0,0,0,0,0,0,0,M,"(1, 2)",3,5,801,0
1999998,80165718,80111,3,2021,1,1,3,7,7,7,45,86,86,86,15,27,27,27,10,13,13,13,0.00,9.67,8.43,8.43,8.43,3,8,8,8,28,136,136,136,14,50,50,50,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",2,4,801,0


# Data Preparation

In [5]:
df = df.dropna()
df = df.drop_duplicates()

df['fulldate'] = pd.to_datetime(df['MONTH'].astype(str) + '-' + df['YEAR'].astype(str))
# change from dtype datetime64[ns] to date time month
df['fulldate'] = df['fulldate'].dt.to_period('M')

df_200k = df.groupby('fulldate')#.apply(lambda x: x.drop(columns=['CUSTOMER_ACCOUNT_NR_MASK']))
df_200k = df.groupby('fulldate').sample(frac=0.01, random_state=101)

#drop customer id column because we want to generalize the model, instead of trying to predict for each customer
df_200k = df_200k.drop(columns=['CUSTOMER_ACCOUNT_NR_MASK'])

In [6]:
df_200k.value_counts('fulldate').sort_index()

fulldate
2021-01     730
2021-02     798
2021-03     871
2021-04     890
2021-05     872
2021-06     929
2021-07     996
2021-08     909
2021-09     487
2021-10     553
2021-11     521
2021-12     615
2022-01    1010
2022-02    1014
2022-03    1038
2022-04    1052
2022-05    1101
2022-06    1116
2022-07    1089
2022-08     969
2022-09     560
2022-10     632
2022-11     591
Freq: M, dtype: int64

In [7]:
numerical_columns = ['CUST_NUM_TRANSACTIONS_30_DAYS', 'CUST_NUM_TRANSACTIONS_90_DAYS',
       'CUST_NUM_TRANSACTIONS_180_DAYS', 'CUST_NUM_TRANSACTIONS_360_DAYS',
       'CUST_NUM_UNIQUE_SUBCAT_30_DAYS', 'CUST_NUM_UNIQUE_SUBCAT_90_DAYS',
       'CUST_NUM_UNIQUE_SUBCAT_180_DAYS', 'CUST_NUM_UNIQUE_SUBCAT_360_DAYS',
       'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS',
       'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS',
       'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS',
       'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS',
       'CUST_AVG_BASKET_SIZE_30_DAYS', 'CUST_AVG_BASKET_SIZE_90_DAYS',
       'CUST_AVG_BASKET_SIZE_180_DAYS', 'CUST_AVG_BASKET_SIZE_360_DAYS',
       'SUBCAT_NUM_TRANSACTIONS_30_DAYS', 'SUBCAT_NUM_TRANSACTIONS_90_DAYS',
       'SUBCAT_NUM_TRANSACTIONS_180_DAYS', 'SUBCAT_NUM_TRANSACTIONS_360_DAYS',
       'SUBCAT_NUM_UNIQUE_CUST_30_DAYS', 'SUBCAT_NUM_UNIQUE_CUST_90_DAYS',
       'SUBCAT_NUM_UNIQUE_CUST_180_DAYS', 'SUBCAT_NUM_UNIQUE_CUST_360_DAYS',
       'CUSTSUBCAT_NUM_TRANSACTIONS_30_DAYS',
       'CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS',
       'CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS',
       'CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS',
       'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS',
       'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS',
       'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS',
       'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS',
       'REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS']

#scaler = MinMaxScaler()
scaler = StandardScaler()
df_200k[numerical_columns] = scaler.fit_transform(df_200k[numerical_columns])

In [8]:
df_200k = df_200k.drop(columns=['QUARTER','SEMESTER','CAT_CD_EXT','SEG_LIFESTYLE_CD', 'YEAR', 'MONTH', 'SUBCAT_CD_EXT'])

In [9]:
# One-hot encoding categorical variables
df_200k = pd.get_dummies(df_200k, columns=['GENDER','SEG_LIFESTAGE_CD', 'FAMILY_MEMBERS'])
df_200k

,CUST_NUM_TRANSACTIONS_30_DAYS,CUST_NUM_TRANSACTIONS_90_DAYS,CUST_NUM_TRANSACTIONS_180_DAYS,CUST_NUM_TRANSACTIONS_360_DAYS,CUST_TOTAL_QTY_BOUGHT_30_DAYS,CUST_TOTAL_QTY_BOUGHT_90_DAYS,CUST_TOTAL_QTY_BOUGHT_180_DAYS,CUST_TOTAL_QTY_BOUGHT_360_DAYS,CUST_NUM_UNIQUE_SUBCAT_30_DAYS,CUST_NUM_UNIQUE_SUBCAT_90_DAYS,CUST_NUM_UNIQUE_SUBCAT_180_DAYS,CUST_NUM_UNIQUE_SUBCAT_360_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS,REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUST_AVG_BASKET_SIZE_30_DAYS,CUST_AVG_BASKET_SIZE_90_DAYS,CUST_AVG_BASKET_SIZE_180_DAYS,CUST_AVG_BASKET_SIZE_360_DAYS,SUBCAT_NUM_TRANSACTIONS_30_DAYS,SUBCAT_NUM_TRANSACTIONS_90_DAYS,SUBCAT_NUM_TRANSACTIONS_180_DAYS,SUBCAT_NUM_TRANSACTIONS_360_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_30_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_90_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_180_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_360_DAYS,SUBCAT_NUM_UNIQUE_CUST_30_DAYS,SUBCAT_NUM_UNIQUE_CUST_90_DAYS,SUBCAT_NUM_UNIQUE_CUST_180_DAYS,SUBCAT_NUM_UNIQUE_CUST_360_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_30_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_30_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_90_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_180_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_360_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS,TARGET,fulldate,GENDER_F,GENDER_M,SEG_LIFESTAGE_CD_1,SEG_LIFESTAGE_CD_2,SEG_LIFESTAGE_CD_3,SEG_LIFESTAGE_CD_4,SEG_LIFESTAGE_CD_5,SEG_LIFESTAGE_CD_6,"FAMILY_MEMBERS_(0, 0)","FAMILY_MEMBERS_(1, 2)","FAMILY_MEMBERS_(3, 8)"
635266,0.40,-0.65,-0.92,-0.97,16,16,16,16,-0.44,-0.98,-1.24,-1.38,-0.55,-0.54,-0.37,-0.26,-0.43,-0.75,-0.85,-0.90,-0.93,-0.37,-0.40,-0.40,-0.38,143,143,143,143,-0.93,-1.14,-1.24,-1.29,-0.13,-0.12,-0.11,-0.10,0,0,0,0,-0.14,-0.13,-0.12,-0.11,0,2021-01,1,0,0,0,0,0,1,0,1,0,0
1128021,0.40,-0.65,-0.92,-0.97,62,62,62,62,1.40,0.30,-0.19,-0.48,-0.67,-0.62,-0.41,-0.28,-0.79,0.69,0.80,0.83,0.84,-0.31,-0.37,-0.38,-0.37,1871,1871,1871,1871,0.56,-0.34,-0.64,-0.80,-0.13,-0.12,-0.11,-0.10,3,3,3,3,-0.14,-0.13,-0.12,-0.11,0,2021-01,1,0,0,0,0,1,0,0,0,0,1
886549,0.77,-0.50,-0.84,-0.92,61,61,61,61,1.14,0.11,-0.34,-0.60,-0.67,-0.62,-0.41,-0.28,-0.43,0.30,0.35,0.36,0.36,-0.37,-0.40,-0.40,-0.38,143,143,143,143,-0.93,-1.14,-1.24,-1.29,-0.13,-0.12,-0.11,-0.10,0,0,0,0,-0.14,-0.13,-0.12,-0.11,0,2021-01,1,0,0,0,0,1,0,0,0,1,0
1053016,-1.08,-1.25,-1.25,-1.15,4,4,4,4,-1.14,-1.47,-1.65,-1.72,2.25,3.82,6.33,7.01,-1.04,-0.52,-0.59,-0.62,-0.64,1.74,0.34,-0.01,-0.16,1804,1809,1809,1809,0.99,-0.11,-0.46,-0.66,-0.13,-0.12,-0.11,-0.10,0,0,0,0,-0.14,-0.13,-0.12,-0.11,0,2021-01,1,0,0,0,0,1,0,0,0,0,1
1970416,0.77,-0.50,-0.84,-0.92,39,39,39,39,-0.61,-1.10,-1.34,-1.46,-0.55,-0.54,-0.37,-0.26,-0.30,-0.44,-0.50,-0.53,-0.54,-0.40,-0.41,-0.40,-0.38,13,13,13,13,-1.16,-1.26,-1.33,-1.36,-0.13,-0.12,-0.11,-0.10,0,0,0,0,-0.14,-0.13,-0.12,-0.11,0,2021-01,1,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774453,-0.34,-0.50,-0.76,-0.46,3,17,23,46,-1.14,-0.98,-1.24,-0.99,0.34,0.41,0.55,0.08,-0.55,-1.02,-0.97,-1.05,-1.07,-0.30,-0.32,-0.31,-0.27,315,652,1241,2824,-0.71,-0.82,-0.78,-0.51,-0.13,-0.12,-0.11,-0.10,0,0,0,0,-0.14,-0.13,-0.12,-0.11,0,2022-11,0,1,0,1,0,0,0,0,0,1,0
1709129,-1.08,-0.65,-0.43,-0.36,11,30,79,108,-1.14,-0.86,-0.34,-0.26,2.25,0.65,0.09,0.04,0.56,-0.69,-0.55,-0.41,-0.60,-0.36,-0.36,-0.35,-0.32,146,435,886,1880,-0.91,-0.92,-0.88,-0.78,-0.13,-0.12,-0.11,-0.10,0,0,0,0,-0.14,-0.13,-0.12,-0.11,0,2022-11,1,0,0,0,0,0,1,0,0,0,1
838640,0.77,-0.05,0.97,1.91,58,100,285,657,0.79,0.30,0.77,1.28,-0.55,-0.14,-0.

In [10]:
df_200k = df_200k.drop('TARGET', axis=1)

# Machine Learning

In [37]:
results_df = pd.DataFrame()

In [62]:
def random_oversampling(x_train,y_train):
    ros = RandomOverSampler(random_state=101)
    X_train, y_train = ros.fit_resample(x_train, y_train)
    
    return X_train, y_train

def roSMOTE(x_train,y_train):
    os = SMOTE(random_state=101)
    X_train, y_train = os.fit_resample(x_train, y_train)
    y_train = y_train.astype(np.float64)
    X_train = y_train.astype(np.float64)
    return X_train, y_train

def random_undersampling(x_train,y_train):
    rus = RandomUnderSampler(random_state=101)
    X_train, y_train = rus.fit_resample(x_train, y_train)
    
    return X_train, y_train

In [53]:
def reg_metrics(pred_model, x_train, x_test, y_train, y_test):
    """ Function takes in training and testing sets, prediction model, 
    and ouputs the below metrics:
    1. R² or Coefficient of Determination.
    2. Adjusted R²
    3. Mean Squared Error(MSE)
    4. Root-Mean-Squared-Error(RMSE).
    5. Mean-Absolute-Error(MAE).
    """
    global r2
    global adj_r2
    global mse
    global rmse
    global mae
    
    # Get predicted values on x_test
    y_pred = pred_model.predict(x_test)

    #1 & 2 Coefficient of Determination (R² & Adjusted R²)
    print("\n\t--- Coefficient of Determination (R² & Adjusted R²) ---")
    r2 = metrics.r2_score(y_pred=y_pred, y_true=y_test)
    adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-x_train.shape[1]-1)

    print(f"R²\t\t: {r2:.2f}")
    print(f"Adjusted R²\t: {adj_r2:.2f}")

    #3 & 4. MSE and RMSE
    print("\n\t--- Mean Squared Error (MSE & RMSE) ---")
   
    mse = metrics.mean_squared_error(y_pred=y_pred, y_true=y_test, squared=True)
    rmse = metrics.mean_squared_error(y_pred=y_pred, y_true=y_test, squared=False)
    
    print(f"MSE\t: {mse:.2f}")
    print(f"RMSE\t: {rmse:.2f}")

    #5. MAE
    print("\n\t--- Mean Absolute Error (MAE) ---")
    mae = metrics.mean_absolute_error(y_pred=y_pred, y_true=y_test)
    print(f"RMSE\t: {mae:.2f}")
    
    # # Return Accuracy
    # #train_acc = "{:.2f}".format(pred_model.score(x_train, y_train)*100)
    # #test_acc = "{:.2f}".format(pred_model.score(x_test, y_test)*100)
    
    # #Convert variables to a rounded format 
    # r2_formatted = "{:.2f}%".format(r2 * 100)
    # adj_r2_formatted = "{:.2f}%".format(adj_r2 * 100)
    # mse_formatted = "{:.2f}".format(mse)
    # rmse_formatted = "{:.2f}".format(rmse)
    # mae_formatted = "{:.2f}".format(mae)
    
    # # Return metrics as a dictionary
    # metrics_dict = {
    #     'R²': r2_formatted,
    #     'Adjusted R²': adj_r2_formatted,
    #     'MSE': mse_formatted,
    #     'RMSE': rmse_formatted,
    #     'MAE': mae_formatted
    # }
    
    #return metrics_dict

In [84]:
def cross_val_reg(df_ML, model, resampling_tecnique, model_name):
    global results_df
    print(f'\t------- {model_name} -------\n')

    timestamps = df_ML['fulldate'].sort_values().unique()

    # store each of the metrics in an array for each one
    rsquared_array = np.array([])
    adjrsquared_array = np.array([])
    mse_array = np.array([])
    rmse_array = np.array([])
    mae_array = np.array([])

    i = 0
    ii = 1
    max_timestamp = timestamps.max() - 2
    while timestamps[i] < max_timestamp:
            print('\n')
            print('-' * 50)
            print(f'\n\n\t---- Start Iteration {ii} ----\n')
            print("\n\t----- Iteration Info -----")
            print(f'\nTreino feito com os meses: {timestamps[i]} até {timestamps[i]+2}') # 11   
            train = df_ML[df_ML['fulldate'].isin([timestamps[i], timestamps[i]+1, timestamps[i]+2])]
            print(f'Teste feito com o mês: {timestamps[i]+3}\n')

            # test and train split
            train_tune = train.groupby('fulldate').sample(frac=0.3, random_state=101)
            X_train_cv = train.drop(['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS','fulldate'], axis=1)
            y_train_cv = train['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'].astype(int)
            X_train_tune = train_tune.drop(['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS','fulldate'], axis=1)
            y_train_tune = train_tune['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'].astype(int) 
                           
            # resampling technique
            X_train_cv, y_train_cv = resampling_tecnique(X_train_cv, y_train_cv)
            X_train_tune, y_train_tune = random_undersampling(X_train_tune, y_train_tune)

            print('X_train_cv shape: ', X_train_cv.shape)
            print('X_train_tune shape: ', X_train_tune.shape)
            
            test = df_ML[df_ML['fulldate'] == timestamps[i]+3] # 12
            
            X_test = test.drop(['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS', 'fulldate'], axis=1)
            y_test = test['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'].astype(int)
            print('X_test shape: ', X_test.shape)
            print('\n\t--------------------------')
            # predictions for metrics
            y_pred = model(X_train_cv, y_train_cv, X_train_tune, y_train_tune, X_test, y_test)
            
            #append rsquared to rsquared_array
            rsquared_array = np.append(rsquared_array, r2)
            #append adjrsquared to ajdrsquared_array
            adjrsquared_array = np.append(adjrsquared_array, adj_r2)
            #append mse to mse_array
            mse_array = np.append(mse_array, mse)
            #append rmse to rmse_array
            rmse_array = np.append(rmse_array, mse)
            #append mae to mae_array
            mae_array = np.append(mae_array, mae)
            
            i += 1
            ii += 1
    else:
        print('\n----------------------------\n\n')
        print(f'R² Mean of all iterations : {np.mean(rsquared_array)}')
        print(f'Adjusted R² Mean of all iterations : {np.mean(adjrsquared_array)}')
        print(f'MSE Mean of all iterations: {np.mean(mse_array)}')
        print(f'RMSE mean of all iterations: {np.mean(rmse_array)}')
        print(f'MAE mean of all iterations: {np.mean(mae_array)}')

        # Append the results to the dataframe
        results_df = results_df.append({
        'Model': model_name,  # Replace 'Model Name' with the actual model name
        'R² Mean': np.mean(rsquared_array),
        'Adjusted R² Mean': np.mean(adjrsquared_array),
        'MSE Mean': np.mean(mse_array),
        'RMSE Mean': np.mean(rmse_array),
        'MAE Mean': np.mean(mae_array)}, ignore_index=True)
        
    return X_train_cv, y_train_cv,X_test, y_test, y_pred, results_df 



In [55]:
def LR(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', LinearRegression())])
    
    param_grid = {}

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf, n_features_to_select=0.5, direction='forward', scoring='r2', cv=None, n_jobs=-1) # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('\n\nNumber of selected features: {}'.format(sfs_backward.n_features_to_select_))
    #print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))
        
    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1, n_jobs=-1)
    LR = Pipeline([('gs', gs)]) 
        
    LR.fit(X_train,y_train)
        
    reg_metrics(LR, X_train, X_test, y_train, y_test)

In [92]:
def ridge(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', Ridge(max_iter=1000, tol=11.830e-06))])  

    param_grid = {
    'clf__alpha': (np.arange(0.001,1.0,0.005))}
    #'clf__alpha': [0.01]}

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf['clf'], n_features_to_select=0.5, direction='forward', scoring='r2', cv=None, n_jobs=-1) # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))
    #print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))
        
    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1, n_jobs=-1)
    RG = Pipeline([('gs', gs)]) 
        
    RG.fit(X_train,y_train)
    
    # get best parameters
    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')

    reg_metrics(RG, X_train, X_test, y_train, y_test)

In [93]:
def lasso(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', Lasso(max_iter=1000, tol=11.830e-06))])  

    param_grid = {
    'clf__alpha': (np.arange(0.001,1.0,0.005))}

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf, n_features_to_select=0.5, direction='forward', scoring='r2', cv=None, n_jobs=-1) # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))
    #print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))
        
    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1, n_jobs=-1)
    LS = Pipeline([('gs', gs)]) 
        
    LS.fit(X_train,y_train)

    # get best parameters
    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')

    reg_metrics(LS, X_train, X_test, y_train, y_test)


In [94]:
def ElasticNetRegression(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
        # ('pca', PCA()),
        ('clf', ElasticNet(max_iter=10000,tol=11.830e-06))])

    param_grid = {
        'clf__alpha': (np.arange(0.001,1.0,0.005))
    }

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(
        clf['clf'], n_features_to_select=0.5, direction='forward', scoring='neg_mean_squared_error', cv=None,
        n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))

    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=2, verbose=1,
                      n_jobs=-1)
    EN = Pipeline([('gs', gs)])

    EN.fit(X_train, y_train)

    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')
    
    reg_metrics(EN, X_train, X_test, y_train, y_test)

In [87]:
def gradientboostingreg(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
    # ('pca', PCA()),
    ('clf', GradientBoostingRegressor())])

    param_grid = {
        'clf__alpha': (np.arange(0.001,1.0,0.005))
    }

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(
        clf['clf'], n_features_to_select=0.5, direction='forward', scoring='r2', cv=None,
        n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))

    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1,
                      n_jobs=-1)
    GB = Pipeline([('gs', gs)])

    GB.fit(X_train, y_train)

    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')
    
    reg_metrics(GB, X_train, X_test, y_train, y_test)

In [97]:
def randomforestreg(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
    # ('pca', PCA()),
    ('clf', RandomForestRegressor())])

    param_grid = {
        'clf__min_samples_split': (range(1,10,1)),
        'clf__min_samples_leaf': (range(1,10,1))
    }

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(
        clf['clf'], n_features_to_select=0.5, direction='forward', scoring='r2', cv=None,
        n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))

    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1,
                      n_jobs=-1)
    GB = Pipeline([('gs', gs)])

    GB.fit(X_train, y_train)

    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')
    
    reg_metrics(GB, X_train, X_test, y_train, y_test)

## Random Oversampling

In [45]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, LR, random_oversampling, 'Linear Regression - OS')

	------- Linear Regression -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (7332, 55)
X_train_tune shape:  (116, 55)
X_test shape:  (890, 55)

	--------------------------


Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits

	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.42
Adjusted R²	: -0.42

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.69
RMSE	: 0.83

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.69


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com o mês: 2021-05

X_train_cv shape:  (7784, 55)
X_train_tune shape:  (136, 55)
X_test shape:  (872, 55)

	--------------------------


Number of selected features: 27
Fitting 2 folds 

In [86]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, ridge, random_oversampling, 'Ridge Regression - OS')

	------- Ridge Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (7332, 55)
X_train_tune shape:  (116, 55)
X_test shape:  (890, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.36
Adjusted R²	: -0.37

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.66
RMSE	: 0.81

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.69


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com o mês: 2021-05


In [95]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, lasso, random_oversampling, 'Lasso Regression - OS')

	------- Lasso Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (7332, 55)
X_train_tune shape:  (116, 55)
X_test shape:  (890, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.25
Adjusted R²	: -0.25

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.60
RMSE	: 0.78

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.62


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com o mês: 2021-05


/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.509e-02, tolerance: 2.100e-03
  model = cd_fast.enet_coordinate_descent(


Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.32
Adjusted R²	: -0.32

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.77
RMSE	: 0.88

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.73


--------------------------------------------------


	---- Start Iteration 4 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-04 até 2021-06
Teste feito com o mês: 2021-07

X_train_cv shape:  (7852, 55)
X_train_tune shape:  (128, 55)
X_test shape:  (996, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
-----------------------------------------

/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.711e-02, tolerance: 1.163e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.685e-01, tolerance: 1.830e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.32

Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.49
Adjusted R²	: -0.50

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.80
RMSE	: 0.89

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.73


--------------------------------------------------


	---- Start Iteration 8 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-08 até 2021-10
Teste feito com o mês: 2021-11

X_train_cv shape:  (5748, 55)
X_train_tune shape:  (68, 55)
X_test shape:  (521, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------

...

Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.32
Adjusted R²	: -0.33

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.80
RMSE	: 0.90

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.74


--------------------------------------------------


	---- Start Iteration 11 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-11 até 2022-01
Teste feito com o mês: 2022-02

X_train_cv shape:  (6224, 55)
X_train_tune shape:  (120, 55)
X_test shape:  (1014, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
---------------------------------------

/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.440e-03, tolerance: 2.044e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.350e-02, tolerance: 1.204e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.87

Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.17
Adjusted R²	: -0.17

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.72
RMSE	: 0.85

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.71


--------------------------------------------------


	---- Start Iteration 15 ----


	----- Iteration Info -----

Treino feito com os meses: 2022-03 até 2022-05
Teste feito com o mês: 2022-06

X_train_cv shape:  (9140, 55)
X_train_tune shape:  (148, 55)
X_test shape:  (1116, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
---------------------------------------

/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.458e-03, tolerance: 1.931e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.458e-03, tolerance: 1.931e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.45

Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.32
Adjusted R²	: -0.32

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.64
RMSE	: 0.80

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.66


--------------------------------------------------


	---- Start Iteration 20 ----


	----- Iteration Info -----

Treino feito com os meses: 2022-08 até 2022-10
Teste feito com o mês: 2022-11

X_train_cv shape:  (6316, 55)
X_train_tune shape:  (84, 55)
X_test shape:  (591, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
-----------------------------------------

In [91]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, ElasticNetRegression, random_oversampling, 'ElasticNet Regression - OS')

	------- ElasticNet Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (7332, 55)
X_train_tune shape:  (116, 55)
X_test shape:  (890, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.28
Adjusted R²	: -0.29

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.62
RMSE	: 0.79

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.63


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com o mês: 202

/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.661e-02, tolerance: 1.775e-02
  model = cd_fast.enet_coordinate_descent(


Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.27
Adjusted R²	: -0.27

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.74
RMSE	: 0.86

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.69


--------------------------------------------------


	---- Start Iteration 4 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-04 até 2021-06
Teste feito com o mês: 2021-07

X_train_cv shape:  (7852, 55)
X_train_tune shape:  (128, 55)
X_test shape:  (996, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
-----------------------------------------

/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.143e-02, tolerance: 9.830e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.744e+00, tolerance: 1.547e-02
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.30

Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.47
Adjusted R²	: -0.47

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.79
RMSE	: 0.89

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.71


--------------------------------------------------


	---- Start Iteration 8 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-08 até 2021-10
Teste feito com o mês: 2021-11

X_train_cv shape:  (5748, 55)
X_train_tune shape:  (68, 55)
X_test shape:  (521, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------

...

Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.23
Adjusted R²	: -0.23

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.65
RMSE	: 0.80

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.64


--------------------------------------------------


	---- Start Iteration 10 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-10 até 2021-12
Teste feito com o mês: 2022-01

X_train_cv shape:  (5084, 55)
X_train_tune shape:  (108, 55)
X_test shape:  (1010, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
---------------------------------------

/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.268e-03, tolerance: 8.659e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.721e-02, tolerance: 1.361e-02
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.26

Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.40
Adjusted R²	: -0.40

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.62
RMSE	: 0.79

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.64


--------------------------------------------------


	---- Start Iteration 19 ----


	----- Iteration Info -----

Treino feito com os meses: 2022-07 até 2022-09
Teste feito com o mês: 2022-10

X_train_cv shape:  (7520, 55)
X_train_tune shape:  (136, 55)
X_test shape:  (632, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
----------------------------------------

/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.314e+00, tolerance: 8.803e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.019e-01, tolerance: 8.803e-03
  model = cd_fast.enet_coordinate_descent(
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.04

Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.30
Adjusted R²	: -0.31

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.60
RMSE	: 0.77

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.61

----------------------------


R² Mean of all iterations : -0.29194191253493146
Adjusted R² Mean of all iterations : -0.29675137397743456
MSE Mean of all iterations: 0.6947984656015436
RMSE mean of all iterations: 0.6947984656015436
MAE mean of all iterations: 0.6784060855168013


In [96]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, gradientboostingreg, random_oversampling, 'Gradient Boosting Regression - OS')

	------- Gradient Boosting Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (7332, 55)
X_train_tune shape:  (116, 55)
X_test shape:  (890, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.01}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.26
Adjusted R²	: -0.26

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.61
RMSE	: 0.78

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.60


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com o m

In [98]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, randomforestreg, random_oversampling, 'Random Forest Regression - OS')

	------- Random Forest Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (7332, 55)
X_train_tune shape:  (116, 55)
X_test shape:  (890, 55)

	--------------------------


## Random Undersampling

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, LR, random_oversampling, 'Linear Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, ridge, random_oversampling, 'Ridge Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, lasso, random_oversampling, 'Ridge Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, ElasticNetRegression, random_oversampling, 'Ridge Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, gradientboostingreg, random_oversampling, 'Gradient Boosting Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, randomforestreg, random_oversampling, 'Random Forest Regression - US')

## SMOTE